In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
import mysql.connector
from mysql.connector import Error
import pandas as pd
from mysql.connector import Error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import seaborn as sns
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Masking, Reshape, Layer, Lambda, Concatenate, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras import backend as be
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta
from keras.layers import BatchNormalization
from keras.regularizers import l2
import matplotlib.pyplot as plt
import LSTM_preprocessing as pre
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
connection = pre.connect_to_database()
if connection:
    # Query merged data
    merged_df = pre.query_merged_crypto_data(connection)
    # converting US bond yield from hourly to yearly under continous compounding assumptions
    merged_df['yield'] = np.exp(merged_df['yield']*365*24)-1

    # Close the connection
    pre.query_quit(connection)

merged_df.head(3)

Connected to MySQL database, MySQL Server version:  8.0.39
MySQL connection is closed


,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,crypto_symbol,date,high,low,close,adj_close,volume,market_cap,coin_supply,yield
0,-50.000000,-50.000000,-50.000000,0.030000,BATUSDT,2020-12-02 14:00:00,0.24,0.24,0.24,0.24,1294545,None,None,0.008552840317602313182744400
1,-50.000000,-50.000000,-50.000000,0.030000,BATUSDT,2020-12-02 15:00:00,0.24,0.24,0.24,0.24,936344,None,None,0.008552840317602313182744400
2,-50.000000,-50.000000,0.013598,0.030000,BATUSDT,2020-12-02 16:00:00,0.24,0.24,0.24,0.24,724626,None,None,0.008552840317602313182744400


### merging with market cap

In [5]:
market_caps = pd.read_csv('market_cap_data.csv')
market_caps = market_caps.rename(columns={
    'timestamp': 'date',
    'token': 'crypto_symbol',
})
market_caps.head(3)

,Unnamed: 0,date,market_cap,crypto_symbol
0,0,2020-12-21 00:00:00,87121414,1INCHUSDT
1,1,2020-12-21 01:00:00,87121414,1INCHUSDT
2,2,2020-12-21 02:00:00,87121414,1INCHUSDT


In [6]:
# Convert 'date' columns to pandas datetime format
merged_df['date'] = pd.to_datetime(merged_df['date'])
market_caps['date'] = pd.to_datetime(market_caps['date'])

# Perform the merge based on 'crypto_symbol' and 'date', keeping only rows in merged_df
merged_df.drop('market_cap', axis = 1, inplace=True)
merged_df = pd.merge(
    merged_df,
    market_caps[['date', 'market_cap', 'crypto_symbol']],
    on=['date', 'crypto_symbol'],
    how='left'
)

merged_df.head(3)

,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,crypto_symbol,date,high,low,close,adj_close,volume,coin_supply,yield,market_cap
0,-50.000000,-50.000000,-50.000000,0.030000,BATUSDT,2020-12-02 14:00:00,0.24,0.24,0.24,0.24,1294545,None,0.008552840317602313182744400,317700843.0
1,-50.000000,-50.000000,-50.000000,0.030000,BATUSDT,2020-12-02 15:00:00,0.24,0.24,0.24,0.24,936344,None,0.008552840317602313182744400,317700843.0
2,-50.000000,-50.000000,0.013598,0.030000,BATUSDT,2020-12-02 16:00:00,0.24,0.24,0.24,0.24,724626,None,0.008552840317602313182744400,317700843.0


In [7]:
merged_df.isna().sum()

lending_rate               0
borrowing_rate             0
utilization_rate           0
stable_borrow_rate         0
crypto_symbol              0
date                       0
high                       0
low                        0
close                      0
adj_close                  0
volume                     0
coin_supply           487675
yield                      0
market_cap              5638
dtype: int64

In [8]:
merged_df[merged_df['market_cap'].isna()].head(3)

,lending_rate,borrowing_rate,utilization_rate,stable_borrow_rate,crypto_symbol,date,high,low,close,adj_close,volume,coin_supply,yield,market_cap
66770,-50.000000,-50.000000,-50.000000,0.030000,KNCUSDT,2020-12-02 14:00:00,1.02,1.01,1.01,1.01,66643,None,0.008552840317602313182744400,NaN
66771,-50.000000,-50.000000,-50.000000,0.030000,KNCUSDT,2020-12-02 15:00:00,1.01,1.00,1.01,1.01,81810,None,0.008552840317602313182744400,NaN
66772,-50.000000,-50.000000,0.333333,0.030000,KNCUSDT,2020-12-02 16:00:00,1.01,1.00,1.00,1.00,61898,None,0.008552840317602313182744400,NaN


In [9]:
market_caps[market_caps['crypto_symbol'] == 'KNCUSDT'].head(3)

,Unnamed: 0,date,market_cap,crypto_symbol
226302,226302,2021-07-26 00:00:00,284924659,KNCUSDT
226303,226303,2021-07-26 01:00:00,284924659,KNCUSDT
226304,226304,2021-07-26 02:00:00,284924659,KNCUSDT


In [10]:
# KNC has wrong data before 2021
merged_df = merged_df[merged_df['market_cap'].notna()]
merged_df.reset_index(inplace = True, drop=True)

In [11]:
merged_df_emb = pre.create_llm_embeddings(merged_df, "crypto_symbol", n_components=15)
merged_df_emb = pre.create_cyclical_encodings(merged_df_emb, "date")

In [12]:
merged_df_emb.columns

Index(['lending_rate', 'borrowing_rate', 'utilization_rate',
       'stable_borrow_rate', 'crypto_symbol', 'date', 'high', 'low', 'close',
       'adj_close', 'volume', 'coin_supply', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine'],
      dtype='object')

In [13]:
# data without NA rows if we needed 
filtered_df = merged_df_emb[(merged_df['borrowing_rate'] != -50)&(merged_df['lending_rate'] != -50)&(merged_df['utilization_rate'] != -50)]
filtered_df['attention'] = filtered_df['volume'] * filtered_df['close']
filtered_df.drop(columns=['stable_borrow_rate','adj_close','coin_supply'], inplace=True)
filtered_df[['lending_rate','borrowing_rate','utilization_rate','close', 'volume']] = filtered_df[['lending_rate','borrowing_rate','utilization_rate','close', 'volume']].astype(float)
filtered_df.reset_index(inplace=True, drop=True)
print(len(filtered_df))
filtered_df.columns

413705


Index(['lending_rate', 'borrowing_rate', 'utilization_rate', 'crypto_symbol',
       'date', 'high', 'low', 'close', 'volume', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention'],
      dtype='object')

In [14]:
# We scale  market cap, attention, yield together as they are comparable
scaler = MinMaxScaler(feature_range=(0, 1))  # Default range is (0, 1)
filtered_df['yield'] = scaler.fit_transform(filtered_df[['yield']])
filtered_df['market_cap'] = scaler.fit_transform(filtered_df[['market_cap']])
filtered_df['attention'] = scaler.fit_transform(filtered_df[['attention']])

### Performing Grid-search on the base arcitecture, for feature importance

In [15]:
comb_1 = ['lending_rate', 'borrowing_rate', 'utilization_rate',
                 'close', 'volume', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention','returns']
comb_2 = ['close', 'volume', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention','returns']
comb_3 = ['lending_rate', 'borrowing_rate', 'utilization_rate',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine']
comb_4 = ['lending_rate', 'borrowing_rate', 'utilization_rate',
                 'close', 'volume',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention','returns']
comb_5 = ['lending_rate', 'borrowing_rate', 'utilization_rate',
                 'close', 'volume', 'yield', 'market_cap',
                   'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention','returns']
comb_6 = ['lending_rate', 'borrowing_rate', 'utilization_rate',
                 'close', 'volume', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'attention','returns']
comb_7 = ['borrowing_rate', 'utilization_rate',
                 'close', 'volume', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention','returns']
comb_8 = ['lending_rate', 'utilization_rate',
                 'close', 'volume', 'yield', 'market_cap',
       'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2',
       'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4',
       'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6',
       'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8',
       'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10',
       'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12',
       'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14',
       'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine',
       'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine',
       'DayofWeek_Cosine', 'attention','returns']

In [16]:
outlier_threshold = 'remove'
input_window = 48
output_window = 48
epoch_num = 100
bacth_size = 200
d1 = 0.1
d2 = 0.05
cell_size = 80
input_columns_list = [comb_1, comb_2, comb_3, comb_4, comb_5, comb_6, comb_7, comb_8]

In [17]:
results_arr = []
for input_columns in input_columns_list:
     try:
          print('preprocessing...')
          input_window = input_window #***
          output_window = output_window #***

          # Initialize empty lists to store aggregated inputs and targets
          all_inputs = []
          all_targets = []

          # looping through symbols
          symbols = filtered_df['crypto_symbol'].unique()
          for s in symbols:
               try:
                    sim_df = filtered_df[filtered_df['crypto_symbol'] == s]
                    sim_df.reset_index(inplace=True, drop=True)
                    # First Loop: Calculate intervals for each column without modifying the DataFrame
                    intervals = {}
                    for column in ['lending_rate', 'borrowing_rate', 'utilization_rate']:
                         lower, upper = pre.calculate_iqr_bounds(sim_df[column],outlier_threshold)
                         intervals[column] = {'lower_bound': lower, 'upper_bound': upper}

                    # getting the returns
                    reduced_df = sim_df.copy()
                    reduced_df['returns'] = pre.calculate_hourly_returns(reduced_df, 'date', 'close')
                    reduced_df = reduced_df[reduced_df['returns'].notna()]
                    reduced_df.reset_index(inplace=True, drop=True)

                    # Second Loop: Filter rows based on the pre-calculated intervals
                    for column in ['lending_rate', 'borrowing_rate', 'utilization_rate']:
                         lower_bound = intervals[column]['lower_bound']
                         upper_bound = intervals[column]['upper_bound']
                         # Apply filtering based on pre-calculated bounds
                         reduced_df = reduced_df[(reduced_df[column] > lower_bound) & (reduced_df[column] < upper_bound)]

                    reduced_df.reset_index(inplace=True, drop=True)

                    # MinMax scaling
                    scaler = MinMaxScaler(feature_range=(0, 1))  # Default range is (0, 1)

                    scaled_df = reduced_df.copy()
                    scaled_df['lending_rate'] = scaler.fit_transform(reduced_df[['lending_rate']])
                    scaled_df['borrowing_rate'] = scaler.fit_transform(reduced_df[['borrowing_rate']])
                    scaled_df['utilization_rate'] = scaler.fit_transform(reduced_df[['utilization_rate']])
                    scaled_df['close'] = scaler.fit_transform(reduced_df[['close']])
                    scaled_df['volume'] = scaler.fit_transform(reduced_df[['volume']])
                    scaled_df['returns'] = scaler.fit_transform(reduced_df[['returns']])

                    inputs, targets = pre.extract_valid_windows_v2(
                         scaled_df,
                         'date', 
                         input_window, output_window, 
                         input_columns, 
                         ['lending_rate','borrowing_rate']
                         )

                    # Append results from the current DataFrame
                    if len(inputs.shape) == 3:
                         all_inputs.append(inputs)
                         all_targets.append(targets)
               except Exception as e:
                    # Handle any other exceptions
                    print(f"Unexpected error in symbol {s}: {e}")

          # Concatenate all inputs and targets into single arrays
          all_inputs = np.concatenate(all_inputs, axis=0) if all_inputs else np.array([])
          all_targets = np.concatenate(all_targets, axis=0) if all_targets else np.array([])

          # Lending rate
          output_1 = all_targets[:, 0].reshape(all_targets.shape[0], 1) 
          # Borrowing rate
          output_2 = all_targets[:, 1].reshape(all_targets.shape[0], 1) 
          total_windows = output_1.shape[0] #***
          print('training ...')
          y_test, y_pred = pre.train_v1(all_inputs, output_1, details=False)
          
          accuracy, precision, recall, f1 = pre.classification_metrics(y_test, y_pred, printed = False)

          new_row = {
               'columns': f'{input_columns}',
               'total_windows':total_windows,
               'input_window':input_window,
               'output_window':output_window, 
               'outlier_threshold':outlier_threshold, 
               'accuracy':accuracy, 
               'precision':precision, 
               'recall':recall, 
               'f1':f1
          }
          results_arr.append(new_row)
          print(new_row)
          
     except Exception as e:
          print(f"Error occurred with {input_columns}")
          print(f"Exception details: {e}")

results = pd.DataFrame(results_arr)

preprocessing...
training ...
1058/1058 [==============================] - 3s 2ms/step
{'columns': "['lending_rate', 'borrowing_rate', 'utilization_rate', 'close', 'volume', 'yield', 'market_cap', 'crypto_symbol_embedding_1', 'crypto_symbol_embedding_2', 'crypto_symbol_embedding_3', 'crypto_symbol_embedding_4', 'crypto_symbol_embedding_5', 'crypto_symbol_embedding_6', 'crypto_symbol_embedding_7', 'crypto_symbol_embedding_8', 'crypto_symbol_embedding_9', 'crypto_symbol_embedding_10', 'crypto_symbol_embedding_11', 'crypto_symbol_embedding_12', 'crypto_symbol_embedding_13', 'crypto_symbol_embedding_14', 'crypto_symbol_embedding_15', 'Month_Sine', 'Month_Cosine', 'Day_Sine', 'Day_Cosine', 'Hour_Sine', 'Hour_Cosine', 'DayofWeek_Sine', 'DayofWeek_Cosine', 'attention', 'returns']", 'total_windows': 169235, 'input_window': 48, 'output_window': 48, 'outlier_threshold': 'remove', 'accuracy': 0.956244275711289, 'precision': 0.9323635935622977, 'recall': 0.9288794896299505, 'f1': 0.930542790711211

In [18]:
results.to_csv('results_feature_importance.csv')

### End of Notebook